In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from pprint import pprint

browser = Browser('chrome')

In [2]:

# Load the web page
url = "https://www.lcbo.com/en/products#t=clp-products&sort=relevancy&layout=card"
browser.visit(url)

html = browser.html
soup=BeautifulSoup(html, 'html.parser')

In [3]:
# To select "English" as preffered language
results = browser.links.find_by_text('English')
results.click()

In [4]:
# This is for loading more products 
max_iteration = 1
iteration = 0

while iteration<= max_iteration:
    # Extract the HTML of the current page
    html = browser.html
    
    # Create a BeautifulSoup object
    soup = BeautifulSoup(html, 'html.parser')
    
    all_links = soup.find_all("div", class_ = 'coveo-product-items')

    links = []
    
    for link in all_links:
        results = link.a['href']
        links.append(results)
        
    # Find the element by ID
    element = soup.find(id='loadMore')
    
    if element is None:
        # Element not found, exit the loop
        break
        
    # Wait for 2 seconds
    time.sleep(1)
    
    
        
    # Click on the element
    browser.find_by_id('loadMore').click()
    
    
    
    iteration += 1
    


In [5]:
len(links)

48

In [6]:
browser = Browser('chrome')


for record in links:
    url = record
    browser.visit(url)
    results = browser.links.find_by_text('English')
    results.click()
    break
    

In [17]:
page_title = soup.find('div', class_ = 'page-title-wrapper product')
test = page_title.find('span', class_ = 'base')
print(test)
# itemprop = soup.find('itemprop')


AttributeError: 'NoneType' object has no attribute 'find'